In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [1]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using SparseArrays
using SparseDiffTools
using SparsityDetection
using LineSearches

ArgumentError: ArgumentError: Package Roots not found in current path:
- Run `import Pkg; Pkg.add("Roots")` to install the Roots package.


In [2]:
Nx = 2
Ne = 8
m = 5
X = zeros(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';

X = ens.S 
idx = [0 0; 0 1; 1 0; 1 1; 1 2; 2 1; 2 2]
Nψ = 7

B = MultiBasis(CstProHermite(6; scaled =true), Nx)

coeff = [-1.0450246575445026;
          0.43100022219193207;
          0.9354686042328957;
         -0.3488477564166211;
          0.21717375788296295;
          0.3992846980946125;
         -1.4098356136623735]


f = ParametricFunction(ExpandedFunction(B, idx, coeff))
R = IntegratedFunction(f);

# F = evaluate(R, X)

F = [    1.1671338483219964;
         0.15492635099770435;
        -0.16262732545001973;
         0.8534477624679682;
        -0.2662915120005553;
         0.6917464124434809;
        -0.8134366100325561;
        -1.1836050661076487]

# Naive approach
Xd = zeros(Ne)
@time for i=1:Ne
    Xd[i] = find_zero(t->evaluate(R, reshape([X[1,i]; t],(2,1)))[1]-F[i], (-5.0, 5.0), Roots.Brent())
end

norm(Xd - X[2,:])<1e-5

MethodError: MethodError: no method matching CstProHermite(::Int64; scaled=true)
Closest candidates are:
  CstProHermite(::Int64) at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/basis.jl:73 got unsupported keyword argument "scaled"

In [8]:
Ne = 500
X = randn(Nx, Ne)
@time F = evaluate(R, X);

  0.000514 seconds (778 allocations: 1.342 MiB)


In [50]:
ψdiag = repeated_evaluate_basis(R.f.f, zeros(Ne));
ψoff = evaluate_offdiagbasis(R.f, vcat(X[1:end-1,:], zeros(1, Ne)));
cacheF= zeros(Ne, Nψ);

In [51]:
output = F - (ψdiag .* ψoff)*R.f.f.coeff;

In [52]:
df_inverse = OnceDifferentiable(functionalf!(cacheF, ψoff, output, R), functionalg!(cacheF, ψoff, output, R), 
                                zeros(Ne), zeros(Ne), Diagonal(zeros(Ne)));

In [53]:
results = nlsolve(df_inverse, zeros(Ne); method = :newton);

In [56]:
@time nlsolve(df_inverse, zeros(Ne); linesearch = LineSearches.BackTracking(), linsolve = (x, A, b) -> copyto!(x, b ./ A.diag), method = :newton)

  0.153618 seconds (218.30 k allocations: 14.336 MiB)


Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [57]:
@btime nlsolve($df_inverse, $zeros(Ne); linsolve = (x, A, b) -> copyto!(x, b./A.diag), method = :newton);

  1.466 ms (3217 allocations: 3.69 MiB)


In [35]:
@btime nlsolve($df_inverse, $zeros(Ne); linsolve = (x, A, b) -> copyto!(x, b./A.diag), method = :newton);

  1.490 ms (3232 allocations: 3.69 MiB)


In [28]:
nlsolve(df_inverse, zeros(Ne); method = :newton)

Results of Nonlinear Solver Algorithm
 * Algorithm: Newton with line-search
 * Starting Point: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [20]:
@btime nlsolve($df_inverse, $zeros(Ne); method = :newton);

  1.490 ms (3232 allocations: 3.69 MiB)


In [22]:
@btime results = nlsolve(df_inverse, zeros(Ne); method = :newton);

  1.495 ms (3232 allocations: 3.69 MiB)


In [144]:
@btime results = nlsolve(df_inverse, zeros(Ne); method = :newton);

  1.776 ms (3312 allocations: 4.32 MiB)


In [143]:
@btime results = nlsolve(df_inverse, zeros(Ne); method = :newton);

  1.768 ms (3312 allocations: 4.32 MiB)


In [125]:
@btime results = nlsolve(only_fj!(functionalfg!($cache, $ψoff, $output, $R)), $zeros(Ne); method = :newton);

  4.296 ms (3330 allocations: 8.17 MiB)


In [75]:
@btime results = nlsolve(only_fj!(functional!($cache, $ψoff, $output, $R)), $zeros(Ne); method = :newton);

  4.507 ms (3330 allocations: 8.17 MiB)


In [81]:
spzeros(2,2)

2×2 SparseMatrixCSC{Float64,Int64} with 0 stored entries

In [62]:
Diagonal(randn(10))

10×10 Diagonal{Float64,Array{Float64,1}}:
 -1.60909    ⋅          ⋅         ⋅        …    ⋅         ⋅         ⋅ 
   ⋅       -0.103464    ⋅         ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅        -0.883652   ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅        0.583092       ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅        …    ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅           -0.765556   ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅        0.257673   ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅         ⋅        0.974417

In [19]:
fieldnames(typeof(results))

(:method, :initial_x, :zero, :residual_norm, :iterations, :x_converged, :xtol, :f_converged, :ftol, :trace, :f_calls, :g_calls)

In [20]:
results.x_converged

false

In [72]:
function f!(F, x)
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

function j!(J, x)
    J[1, 1] = x[2]^3-7
    J[1, 2] = 3*x[2]^2*(x[1]+3)
    u = exp(x[1])*cos(x[2]*exp(x[1])-1)
    J[2, 1] = x[2]*u
    J[2, 2] = u
end

nlsolve(f!, j!, [ 0.1; 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818049096324184e-16, 1.0000000000000002]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In [68]:
?OnceDifferentiable

search: OnceDifferentiable



No documentation found.

`NLSolversBase.OnceDifferentiable` is of type `UnionAll`.

# Summary

```
struct UnionAll <: Type{T}
```

# Fields

```
var  :: TypeVar
body :: Any
```

# Supertype Hierarchy

```
UnionAll <: Type{T} <: Any
```


In [4]:
using NLsolve
using SparseArrays
using SparseDiffTools
using SparsityDetection

Ne = 200

function f!(F, x)
    @inbounds for (i, xi) in enumerate(x)
        F[i] = exp(-xi^2/(2*i))*erf(xi)
    end 
    nothing
end

function g_diag!(J, x)
    @inbounds for (i, xi) in enumerate(x)
        J.diag[i] = (-xi/i)*exp(-xi^2/(2*i))*erf(xi) + exp(-xi^2/(2*i))*(2/√π)*exp(-xi^2)
    end 
    nothing
end

function g_sparse!(J, x)
    @inbounds for (i, xi) in enumerate(x)
        J[i,i] = (-xi/i)*exp(-xi^2/(2*i))*erf(xi) + exp(-xi^2/(2*i))*(2/√π)*exp(-xi^2)
    end 
end

df_diag = OnceDifferentiable(f!, g_diag!, randn(Ne), rand(Ne), Diagonal(zeros(Ne)))
df_sparse = OnceDifferentiable(f!, g_sparse!, randn(Ne), rand(Ne), spzeros(Ne,Ne))


F = rand(Ne);

@btime r_diag = nlsolve($df_diag, $F; linsolve = (x, A, b) -> copyto!(x, b ./ A.diag), method = :newton);
@btime r_sparse = nlsolve($df_sparse, $F; method = :newton);

  114.696 μs (88 allocations: 91.44 KiB)
  221.794 μs (488 allocations: 298.63 KiB)


In [6]:
input = zeros(Ne)
output = similar(input)
sparsity_pattern = ones(Ne)
jac = Float64.(sparse(sparsity_pattern));

In [7]:
jac

200-element SparseVector{Float64,Int64} with 200 stored entries:
  [1  ]  =  1.0
  [2  ]  =  1.0
  [3  ]  =  1.0
  [4  ]  =  1.0
  [5  ]  =  1.0
  [6  ]  =  1.0
  [7  ]  =  1.0
  [8  ]  =  1.0
  [9  ]  =  1.0
  [10 ]  =  1.0
         ⋮
  [190]  =  1.0
  [191]  =  1.0
  [192]  =  1.0
  [193]  =  1.0
  [194]  =  1.0
  [195]  =  1.0
  [196]  =  1.0
  [197]  =  1.0
  [198]  =  1.0
  [199]  =  1.0
  [200]  =  1.0

In [100]:
@btime r = nlsolve($df, $F, method = :newton);

  243.315 μs (534 allocations: 326.47 KiB)
